### This notebook precomputes the integral (see Notebook 2) that will otherwise be difficult to evaluate. The precomputed values will be loaded in Notebook 2, and we make interpolation from the precomputed values.

In [2]:
import scipy as sp
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
import scipy.integrate as integrate


from astropy import units as u
from astropy.coordinates import SkyCoord

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
filename = "precomputed_integral_values.txt" # where the precomputed values will be stored


We need to consider the distance selection effect and spatial/regional selection effect.

In [5]:
# Distance Selection Effect, MIN and MAX distance of our catalog
d_min = 13.0896
d_max = 68.2796

# Region Selection Effect, DES footprint coverage
ra_min, ra_max, dec_min, dec_max = -60, 100, -70, 10 


In [6]:
# constant values
r_0=8.3 # the distance from the Sun to the Galactic Center
r_smooth = 1 # makes the integral OK at 0...

In [1]:

def survey_function(l1, b1):
    galatic_coord = SkyCoord(l=l1*180/np.pi*u.deg, b=b1*180/np.pi*u.deg, frame='galactic')
    icrs_coord = galatic_coord.transform_to('icrs')
    ra,dec = icrs_coord.ra.to(u.deg), icrs_coord.dec.to(u.deg)
    if ra >180*u.deg:
        ra = (180*u.deg)-ra

    if ra <ra_min*u.deg or ra >ra_max*u.deg:
        return 0
    elif dec <dec_min*u.deg or dec >dec_max*u.deg:
        return 0
    else:
        return 1


# tranforming each l and b into ra and dec using astropy during the integral evaluation is extremely slow. So transformed the grid for 
# l and b in advance (same grids as used in the later integral evaluation) and select those (l,d) which correspond to (ra, dec) inside
# DES footprint. 
n = 50
b_s = np.linspace(-0.5*np.pi, 0.5*np.pi, n+1)
l_s = np.linspace(0*np.pi, 2*np.pi, n+1)
selection=np.zeros((len(l_s),b_s.shape[0]))
valid_set = []
for i in range(len(l_s)):
    for j in range(len(b_s)):
        selected=survey_function(l_s[i], b_s[j])
        selection[i,j] = selected
        if selected==1:
            valid_set.append((l_s[i], b_s[j]))


In [5]:
# functions to compute the integral

def func_3(b,l,d, alpha):
    """Return the integrand given l,b,d."""
    r=np.sqrt(r_0 ** 2 + d ** 2 - 2 * r_0 * d * np.cos(b) * np.cos(l))
    if r<20:
        return 0
    return (d ** 2) * np.cos(b) * np.power(r + r_smooth,
                                           -1 * alpha) * ((l,b) in valid_set)


# use a 3-fold trapezoidal rule
def func_2_inner(b,l, alpha):
    """For fixed b and l, evaluate the integral on the third variable 'd'.
    To evaluate the inner integral, recursively call trapezoidal rule on the integral."""

    int_range = d_max - d_min
    n = int(int_range*2)
    d_s = np.linspace(d_min, d_max, n+2)
    h = int_range / (n+1) 
    
    f = [func_3(b,l,d, alpha) for d in d_s]
    f = np.array(f)
    I_trap = (h / 2) * (f[0] + f[n]+2 * sum(f[1:n]))
    return I_trap


def func_1_inner(l, alpha):
    """For fixed l, evaluate integral on the second variable 'b'.
    To evaluate the inner integral, recursively call trapezoidal rule on the integral."""
    n = 50
    b_s = np.linspace(-0.5 * np.pi, 0.5 * np.pi, n+1)
    h = np.pi / n
    f = [func_2_inner(b,l,alpha) for b in b_s]
    f = np.array(f)
    I_trap = (h / 2) * (f[0] + f[n]+2 * sum(f[1:n]))
    return I_trap


def func_inner(alpha):
    """Return the integral.
    Evaluate on the first variable 'l'.
    """
    n = 50
    l_s = np.linspace(0, 2 * np.pi, n+1)
    h = 2 * np.pi / n
    f = [func_1_inner(l, alpha) for l in l_s]
    f = np.array(f)
    I_trap = (h / 2) * (f[0] + f[n]+2 * sum(f[1:n]))
    return I_trap

We evaluate the integral at a grid of 50 evenly-spaced points from 2 to 7 for parameter $\alpha$.

In [6]:
alphas_inner = np.linspace(2,7, 50)
interpolation_inner = np.zeros(shape=(len(alphas_inner)))
for i in range(len(alphas_inner)):
    interpolation_inner[i] = func_inner(alphas_inner[i])
    print(f'alpha = {alphas_inner[i]}, int={interpolation_inner[i]}')

alpha = 2.0, int=50.127154404773165
alpha = 2.1020408163265305, int=34.10332634965969
alpha = 2.204081632653061, int=23.22864642169163
alpha = 2.306122448979592, int=15.840225701331262
alpha = 2.4081632653061225, int=10.814722227381456
alpha = 2.510204081632653, int=7.392503389807169
alpha = 2.612244897959184, int=5.059351010781693
alpha = 2.7142857142857144, int=3.466805429322822
alpha = 2.816326530612245, int=2.3784774702517146
alpha = 2.9183673469387754, int=1.6338279628617565
alpha = 3.020408163265306, int=1.1237076585115013
alpha = 3.122448979591837, int=0.7738224642590759
alpha = 3.2244897959183674, int=0.5335446319347898
alpha = 3.326530612244898, int=0.3683335374363532
alpha = 3.428571428571429, int=0.254596088774835
alpha = 3.5306122448979593, int=0.17619769712068917
alpha = 3.63265306122449, int=0.1220911196260565
alpha = 3.7346938775510203, int=0.08470320260739225
alpha = 3.836734693877551, int=0.05883602460188762
alpha = 3.938775510204082, int=0.04091755474407851
alpha = 4.

In [7]:
# store the file
np.savetxt(filename, interpolation_inner)